In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
assert tf.config.list_physical_devices('GPU')

Using TensorFlow backend.


In [26]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)
    layer = layers.add([layer,
                         layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer))
    ]
    model = tf.keras.Model(inputs=[input_text], outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # tf.keras.utils.plot_model(model, to_file='model.png')
    return model

model = build_model()

def fit(data, learning_rates):
    x  = data.normalized_texts
    vx = data.normalized_validation
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,      'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D': data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=len(learning_rates),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
          ]
    )

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 64, 110)      4840        input_7[0][0]                    
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional (None, 64, 220)      582560      embedding_6[0][0]                
__________________________________________________________________________________________________
bidirectional_13 (Bidirectional (None, 64, 220)      776160      bidirectional_12[0][0]           
____________________________________________________________________________________________

In [3]:
def load_data(source, maxlen=MAXLEN):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, 0.1, maxlen=maxlen, shuffle=True)

In [4]:
data_rabanit = load_data(['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt', 'tomer_dvora.txt'])

In [5]:
data_pre_modern = load_data(['elef_layla.txt', 'bialik', 'shaul_tchernichovsky', 'breslev.txt', 'itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson',
                             'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri', 'atar_hashabat.txt', 'ali_baba.txt'])

In [12]:
data_modern = load_data(['forums', 'newspapers', 'wiki', 'blogs', 'adamtsair.txt', 'katarsis.txt'])  # , 'imagination.txt', 'sipurim.txt', 'ricky.txt'

In [27]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, [3e-3, 3e-4])
model.save_weights('./checkpoints/rabanit')

Train on 76377 samples, validate on 8487 samples
Epoch 1/2
76377/76377 [==============================] - 91s 1ms/sample - loss: 0.3397 - N_loss: 0.2651 - D_loss: 0.0657 - S_loss: 0.0089 - N_accuracy: 0.9073 - D_accuracy: 0.9745 - S_accuracy: 0.9975 - val_loss: 0.1583 - val_N_loss: 0.1195 - val_D_loss: 0.0356 - val_S_loss: 0.0034 - val_N_accuracy: 0.9592 - val_D_accuracy: 0.9870 - val_S_accuracy: 0.99932877 - D_loss: 0.0702 - S_loss: 0.0098 - N_accuracy: 0.8992 - D_accuracy: 0.9726 - S_accuracy - ETA: 12s - loss: 0.3671 - N_loss: 0.2872 - D_loss: 0.0701 - S_loss - ETA: 9s - loss: 0.3604 - N_loss: 0.2818 - D_loss: 0.0691 - S_loss: 0.0096 - N_accuracy: 0.9013 - D_accuracy: 0 - ETA: 7s - loss: 0.3560 - N_loss: 0.2783 - D_loss: 0.0683 - S_loss: 0.0094 - N_accuracy: 0.902 - ETA: 4s - loss: 0.3494 - N_loss: 0.2730 - D_loss: 0.0673 - S_loss: 0.0092 - N_acc - ETA: 0s - loss: 0.3413 - N_loss: 0.2664 - D_loss: 0.0660 - S_loss: 0.0089 - N_accuracy: 0.9068 - D_accuracy: 0.9744 - S_accu
Epoch 2/2
7

In [28]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, [3e-3, 3e-4])
model.save_weights('./checkpoints/pre_modern')

Train on 84096 samples, validate on 9344 samples
Epoch 1/2
84096/84096 [==============================] - 89s 1ms/sample - loss: 0.2806 - N_loss: 0.2123 - D_loss: 0.0618 - S_loss: 0.0064 - N_accuracy: 0.9277 - D_accuracy: 0.9751 - S_accuracy: 0.9982 - val_loss: 0.2207 - val_N_loss: 0.1633 - val_D_loss: 0.0520 - val_S_loss: 0.0053 - val_N_accuracy: 0.9454 - val_D_accuracy: 0.9791 - val_S_accuracy: 0.9986 - S_loss: 0.0081 - N_accuracy: 0.8993 - D_accuracy: 0.9686 - S_accu - ETA: 1: - ETA: 1:22 - loss: 0.3490 - N_loss: 0.2682 - D_loss: 0.0730 - S_loss: 0.0078 - N_accuracy: 0.9087 - D_accuracy: - ETA: 1:18 - loss: 0.3441 - N_loss: 0.2643 - D_loss: 0.0722 - S_loss: 0.0076 - N_accuracy: 0.9099 - D_accuracy - ETA: 1:14 - loss: 0.3398 - N_loss: 0.2606 - D_loss: 0.0716 - S_loss:  - ETA: 1:07 - loss: 0.3307 - N_loss: 0.2531 - D_loss: 0.0702 - S_loss: 0.0074  - ETA: 1:01 - loss: 0.3245 - N_loss: 0.2479 - D_loss: 0.0693 - S_loss: 0.0073 - N_accuracy: 0.9 - ETA: 7s - loss: 0.2841 - N_loss: 0.2152 -

In [29]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, [3e-3, 3e-4])

Train on 11692 samples, validate on 1300 samples
Epoch 1/2
11692/11692 [==============================] - 13s 1ms/sample - loss: 0.2441 - N_loss: 0.1825 - D_loss: 0.0563 - S_loss: 0.0053 - N_accuracy: 0.9419 - D_accuracy: 0.9783 - S_accuracy: 0.9988 - val_loss: 0.1968 - val_N_loss: 0.1476 - val_D_loss: 0.0436 - val_S_loss: 0.0050 - val_N_accuracy: 0.9560 - val_D_accuracy: 0.9831 - val_S_accuracy: 0.9990
Epoch 2/2
11692/11692 [==============================] - 13s 1ms/sample - loss: 0.1583 - N_loss: 0.1163 - D_loss: 0.0384 - S_loss: 0.0035 - N_accuracy: 0.9642 - D_accuracy: 0.9854 - S_accuracy: 0.9992 - val_loss: 0.1753 - val_N_loss: 0.1315 - val_D_loss: 0.0389 - val_S_loss: 0.0044 - val_N_accuracy: 0.9612 - val_D_accuracy: 0.9850 - val_S_accuracy: 0.9990_accuracy:  - ETA: 5s - loss: 0.1612 - N_loss: 0.1186 - D_loss: 0.0390 - S_loss: 0.0035 - N_accuracy: 0.963 - ETA: 2s - loss: 0.1588 - N_loss: 0.1165 - D_loss: 0.0387 - S_loss: 0.0036 - N_accuracy: 0.9640 - D_ac - ETA: 0s - loss: 0.1581

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [10]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [30]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(load_data(['newspapers'], maxlen=MAXLEN), 0)

מצוי:  לְכִיוּוּן הַמַּתְאִים. תְּנוּ לָהֶם עוֹד חוֹפֶשׁ לִפְנֵי הַתִּיקּוּן וְהֵם יִשָׂאֲפוּ לְסַבְסֵב אֶת
רצוי:  לַכִּיוּוּן הַמַּתְאִים. תְּנוּ לָהֶם עוֹד חוֹפֶשׁ לִפְנֵי הַתִּיקּוּן וְהֵם יִשְׁאֲפוּ לְסַבְסֵב אֶת

מצוי:  לַפִיהָ הַגֶּנֵטִיקָה שֶׁיָּרַשְׁנוּ הִיא הַגּוֹרֵם הַמֶּרְכָּזִי שֶׁקּוֹבֵעַ אֶת עָתִידֵנוּ הַבְּרִיאוּתִי,
רצוי:  לְפִיהָ הַגֵּנֵטִיקָה שֶׁיָּרַשְׁנוּ הִיא הַגּוֹרֵם הַמֶּרְכָּזִי שֶׁקּוֹבֵעַ אֶת עֲתִידֵנוּ הַבְּרִיאוּתִי,

מצוי:  זָקוּק לָהּ בְּמַצַּב הַחֵירוּם הַנּוֹכְחִי". נַזְכִּיר כִּי בְּיוֹם שְׁנֵי הָאַחֲרוֹן הוּפְסַק
רצוי:  זָקוּק לָהּ בְּמַצַּב הַחֵירוּם הַנּוֹכְחִי". נַזְכִּיר כִּי בַּיּוֹם שֵׁנִי הָאַחֲרוֹן הוּפְסַק

מצוי:  טִיפָּה נִלְחַצְתִּי: זְכוּת גְּדוֹלָה וּסְגִירַת מַעְגָּל. הִסְתַּכַּלְתִּי עַל הָאוּלָם. יָשְׁבָה שָׁם
רצוי:  טִיפָּה נִלְחַצְתִּי: זְכוּת גְּדוֹלָה וּסְגִירַת מַעְגָּל. הִסְתַּכַּלְתִּי עַל הָאוּלָם. יָשְׁבָה שָׁם

מצוי:  עִם שְׁתֵּי כּוֹסוֹת תָּה. "אֲנִי מִצְטַעֵר שֶׁזֶּה כָּכָה, פָּשׁוּט הַמִּטְבָּח שֶׁלָּנוּ בַּחוּץ".
רצוי:  ע